In [ ]:
import yt_dlp
import cv2
import math

# Function for vehicle detection using any Cascade Classifier
def detect_vehicles(frame):
    car_cascade = cv2.CascadeClassifier("C:\\Users\\kunal\\Downloads\\fest\\cars.xml")
    bike_cascade = cv2.CascadeClassifier("C:\\Users\\kunal\\Downloads\\fest\\two_wheeler.xml")
    bus_cascade = cv2.CascadeClassifier("C:\\Users\\kunal\\Downloads\\fest\\Bus_front.xml")
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    cars = car_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(25, 25))
    bikes = bike_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(25, 25))
    buses = bus_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(25, 25))

    vehicles = []
    vehicles.extend(cars)
    vehicles.extend(bikes)
    vehicles.extend(buses)

    return vehicles, bikes, buses

direct_url = 'https://www.youtube.com/watch?v=iJZcjZD0fw0'

# Use yt-dlp to download the video
ydl_opts = {
    'format': 'mp4',
    'outtmpl': 'video.mp4',
}
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([direct_url])

# Open the downloaded video using OpenCV
cap = cv2.VideoCapture('video.mp4')

count = 0
center_points_prev_frame = []
tracking_objects = {}
track_id = 0
output_size = (1200, 900)

while True:
    ret, frame = cap.read()
    count += 1
    if not ret:
        break

    frame = cv2.resize(frame, output_size)
    center_points_cur_frame = []

    vehicles, bikes, buses = detect_vehicles(frame)

    for vehicle_id, vehicle_info in list(tracking_objects.items()):
        if vehicle_info['frame'] + 8 < count:
            tracking_objects.pop(vehicle_id)
        else:
            pt = vehicle_info['position']
            cv2.circle(frame, pt, 5, (0, 0, 255), -1)
            cv2.putText(frame, f"{vehicle_id}", (pt[0], pt[1] - 7), 0, 1, (0, 0, 255), 2)

    for (x, y, w, h) in vehicles:
        cx = int(x + w // 2)
        cy = int(y + h // 2)
        center_points_cur_frame.append((cx, cy))

        if (x, y, w, h) in buses:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        elif (x, y, w, h) in bikes:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        else:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)

        match_existing = False
        for vehicle_id, vehicle_info in tracking_objects.items():
            distance = math.hypot(vehicle_info['position'][0] - cx, vehicle_info['position'][1] - cy)
            if distance < 20:
                tracking_objects[vehicle_id]['position'] = (cx, cy)
                tracking_objects[vehicle_id]['frame'] = count
                match_existing = True
                break

        if not match_existing:
            tracking_objects[track_id] = {'position': (cx, cy), 'frame': count}
            track_id += 1

    grid_density = []
    for i in range(16):
        for j in range(16):
            cell_x = int((frame.shape[1] / 16) * i)
            cell_y = int((frame.shape[0] / 16) * j)
            cell_w = int(frame.shape[1] / 16)
            cell_h = int(frame.shape[0] / 16)

            cell_area = cell_w * cell_h
            cell_vehicles = [(x, y, w, h) for (x, y, w, h) in vehicles if
                              cell_x <= (x + w // 2) <= (cell_x + cell_w) and cell_y <= (y + cell_h) and (x, y, w, h) not in tracking_objects.values()]

            cell_density = (len(cell_vehicles) / cell_area) * 100
            grid_density.append(cell_density)

            cv2.rectangle(frame, (cell_x, cell_y), (cell_x + cell_w, cell_y + cell_h), (255, 255, 255), 1)
            cv2.putText(frame, f"{cell_density:.2f}%", (cell_x + 5, cell_y + 15), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)

    cv2.imshow('Video', frame)

    key = cv2.waitKey(30)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()


[youtube] Extracting URL: https://www.youtube.com/watch?v=iJZcjZD0fw0
[youtube] iJZcjZD0fw0: Downloading webpage
[youtube] iJZcjZD0fw0: Downloading ios player API JSON
[youtube] iJZcjZD0fw0: Downloading player 250a2ff7


         n = PNfuTTP_I6YojMuJi_WR46 ; player = https://www.youtube.com/s/player/250a2ff7/player_ias.vflset/en_US/base.js
         n = 7vUErwL012JkX3psTkbH4n ; player = https://www.youtube.com/s/player/250a2ff7/player_ias.vflset/en_US/base.js


[youtube] iJZcjZD0fw0: Downloading m3u8 information
[info] iJZcjZD0fw0: Downloading 1 format(s): 609
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 165
[download] Destination: video.mp4
[download] 100% of  112.53MiB in 00:02:14 at 859.88KiB/s                  
